In [1]:
import numpy as np
import pandas as pd
import random
from random import randint
import string
from pyspark.sql.types import *
from pyspark.sql.functions import when
from pyspark.ml.feature import Imputer
from datetime import datetime, timedelta
from pyspark import SparkContext


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("sourcedata").getOrCreate()

In [5]:
def getRandomDates():
    start = datetime.now()
    end = start + timedelta(days=700)
    random_date = (start - (end - start) * random.random()).strftime('%Y-%m-%d')
    return random_date

In [6]:
def get_random_string(length):
    # choose from all lowercase letter
    letters = string.ascii_lowercase
    result_str = ''.join(random.choice(letters) for i in range(length))
    return result_str

In [7]:
def build_schema(dataset_date):
    print("Build and return a schema to use for the sample data.")
    schema = StructType(
        [
            StructField("account_id",  StringType(),  True),
            StructField("account_open_dt",      StringType(),  True),
            StructField("account_id_type", StringType(),  True),
            StructField("acct_hldr_primary_addr_state", StringType(),  True),
            StructField("acct_hldr_primary_addr_zip_cd",    StringType(),  True),
            StructField("acct_hldr_first_name", StringType(),  True),
            StructField("acct_hldr_last_name", StringType(),  True),
            StructField("dataset_date",  StringType(),  True)
        ]
    )
    return schema

In [13]:
from pyspark import SparkContext 
import os

def build_data(dataset_date, cnt):
    rec = []
    for i in range(1,rec_cnt):
        account_id =  random.choice(["SV","CK","PV"])+str(randint(27618,189765))
        account_open_dt = getRandomDates()
        acct_hldr_primary_addr_zip_cd=str(randint(23456,98776))
        acct_hldr_primary_addr_state = random.choice(['AL','AK','AZ','AR','CA','CO','CT','DE','DC',\
                                                      'FL','GA','HI','ID','IL','IN','IA','KS','KY','LA',\
                                                      'ME','MD','MA'])
        account_id_type = random.choice(["Saving","Checkin","Private"])
        acct_hldr_first_name=get_random_string(10)
        acct_hldr_last_name=get_random_string(6)
        rec.append(account_id + "~" +
                   account_open_dt + "~" +
                   account_id_type + "~" +
                   acct_hldr_primary_addr_state + "~" +
                   acct_hldr_primary_addr_zip_cd + "~" +
                   acct_hldr_first_name + "~" +
                   acct_hldr_last_name + "~" +
                   dataset_date)
    return (rec)






rec_cnt=1001
dataset_dates=["2022-01-01","2022-01-02","2022-01-03","2022-01-04","2022-01-05","2022-01-06","2022-01-08","2022-01-00","2022-01-07","2022-01-10"]
for d in dataset_dates:
    data_dir="data/cards_ingest/cards_account_ingest/"
    file_name=data_dir+"cards_account_ingest_"+d+".csv"
    columns=['account_id','account_open_dt','account_id_type','acct_hldr_primary_addr_state',\
                'acct_hldr_primary_addr_zip_cd','acct_hldr_first_name','acct_hldr_last_name','dataset_date']
    sc = spark.sparkContext
    hist_data = build_data(d, int(rec_cnt))
    histrawInput = sc.parallelize(hist_data)
    histrawInputSplit = histrawInput.map(lambda x: x.split("~"))
    hist_df = spark.createDataFrame(histrawInputSplit, schema=build_schema(d))
    hist_df.toPandas().to_csv(file_name,index=False)
#






Build and return a schema to use for the sample data.


Py4JJavaError: An error occurred while calling o39.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (DESKTOP-KE4IQMD executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadChec

In [16]:
# hist_df.toPandas().to_csv(file_name,index=False)